In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
import random
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file_path = '/content/cleaned_dataMA.xlsx'

In [6]:
!pip install transformers

In [7]:
import pandas as pd
data = pd.read_excel(file_path)

In [8]:
data.head()

,egyption_Text,english_Text,category,sub_category
0,الأمير الصغير,little prince,Novels,the-little-prince
1,أنطوان دي سانت إكزوبيري,Antoine De Saint-Exuper,Novels,the-little-prince
2,الفصل الأول,Chapter 1,Novels,the-little-prince
3,في مره، لما كان عندي ست سنين، شفت صوره روعه، ف...,"six years old saw magnificent picture book , c...",Novels,the-little-prince
4,وكانت عباره عن تعبان من نوع البوا بيبلع فهد.,picture boa constrictor act swallowing animal .,Novels,the-little-prince


In [9]:
data.shape

(23539, 4)

In [15]:

# load the arat5 tokenizer and model (Arabic dialect translation)
dialect_tokenizer = AutoTokenizer.from_pretrained("PRAli22/arat5-arabic-dialects-translation")
dialect_model = AutoModelForSeq2SeqLM.from_pretrained("PRAli22/arat5-arabic-dialects-translation")
# load the helsinki tokenizer and model (Arabic to English translation)
ar_to_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
#ar_to_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
# Relative path to the model files
model_dir = 'Finetuned Helsinki Model (Saradar)'
model_path = '/content/drive/My Drive/{}/'.format(model_dir)
ar_to_en_model=AutoModelForSeq2SeqLM.from_pretrained(model_path)



In [16]:
# function to translate dialect to English
def translate_dialect_to_english(dialect_text):
    # tokenize input (dialect text )
    input_ids = dialect_tokenizer(dialect_text, return_tensors="pt").input_ids
    # translation from dialect to standard Arabic
    outputs = dialect_model.generate(input_ids)
    standard_arabic_text = dialect_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # translate standard Arabic text to English
    input_ids = ar_to_en_tokenizer(standard_arabic_text, return_tensors="pt").input_ids
    outputs = ar_to_en_model.generate(input_ids)
    english_translation = ar_to_en_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return english_translation

In [17]:
def translate_to_english_helsinki(ar_text):
    # tokenize input (dialect text)
    input_ids = dialect_tokenizer(ar_text, return_tensors="pt").input_ids
    # translate dialect to English using Helsinki model
    input_ids = ar_to_en_tokenizer(ar_text, return_tensors="pt").input_ids
    outputs = ar_to_en_model.generate(input_ids)
    english_translation = ar_to_en_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return english_translation

In [18]:
def semantic_similarity(text1, text2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return similarity

In [19]:
def translate_dialect_to_standard_arabic(dialect_text):
    input_ids = dialect_tokenizer(dialect_text, return_tensors="pt").input_ids
    outputs = dialect_model.generate(input_ids)
    standard_arabic_text = dialect_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return standard_arabic_text

In [22]:
ar_translated_dir = '/content/drive/My Drive/'

# first approach (using finetuned model)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
from tqdm import tqdm

chunk_size = 700
total_rows = len(data)
num_chunks = total_rows // chunk_size + (1 if total_rows % chunk_size != 0 else 0)
translated_data = []
# Convert english_Text column to string type
data['english_Text'] = data['english_Text'].astype(str)

# Now, iterate over the chunks
for chunk_index in tqdm(range(num_chunks)):
    start_index = chunk_index * chunk_size
    end_index = min((chunk_index + 1) * chunk_size, total_rows)
    chunk_data = data.iloc[start_index:end_index]
    for index, row in chunk_data.iterrows():
        dialect_text = str(row['egyption_Text'][:4000])
        standard_arabic_text = translate_dialect_to_standard_arabic(dialect_text)

        # translate standard Arabic text to English using Helsinki model
        helsinki_translation = translate_to_english_helsinki(standard_arabic_text)

        # calculate BLEU score for the Helsinki translation
        reference_translation = row['english_Text'][:4000].split()
        translated_translation = helsinki_translation.split()
        bleu_helsinki = sentence_bleu([reference_translation], translated_translation)

        # calculate BLEU score for the Egyptian text in the Helsinki model with the reference English text
        helsinki_bleu_egyptian = sentence_bleu([reference_translation], translate_to_english_helsinki(dialect_text).split())

        # append the data to the list
        translated_data.append({
            'egyption_Text': dialect_text,
            'standard_arabic_Text': standard_arabic_text,
            'english_Text': row['english_Text'][:4000],
            'bleu_score_helsinki': bleu_helsinki,
            'bleu_score_helsinki_egyptian': helsinki_bleu_egyptian
        })


# DataFrame from the collected data
translated_df = pd.DataFrame(translated_data)
file_path = ar_translated_dir + f'translated_texts_all_chunks.csv'
translated_df.to_csv(file_path, index=False, encoding='utf-8')


  6%|▌         | 2/34 [1:19:56<21:08:56, 2379.27s/it]

In [26]:
# function to translate dialect to English using AraT5
def translate_dialect_to_english_arat5(dialect_text):
    # tokenize input (dialect text)
    input_ids = dialect_tokenizer(dialect_text, return_tensors="pt").input_ids
    # translation from dialect to standard Arabic
    outputs = dialect_model.generate(input_ids)
    standard_arabic_text = dialect_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # translate standard Arabic text to English
    input_ids = ar_to_en_tokenizer(standard_arabic_text, return_tensors="pt").input_ids
    outputs = ar_to_en_model.generate(input_ids)
    english_translation = ar_to_en_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return english_translation

In [27]:
def translate_dialect_to_english(dialect_text):
    # tokenize input (dialect text)
    input_ids = dialect_tokenizer(dialect_text, return_tensors="pt").input_ids
    # translation from dialect to standard Arabic
    outputs = dialect_model.generate(input_ids)
    standard_arabic_text = dialect_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # translate standard Arabic text to English
    input_ids = ar_to_en_tokenizer(standard_arabic_text, return_tensors="pt").input_ids
    outputs = ar_to_en_model.generate(input_ids)
    english_translation = ar_to_en_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # calculate semantic similarity between translated English text and original dialect text
    similarity_score = semantic_similarity(english_translation, dialect_text)
    # check if semantic similarity is below the threshold
    if similarity_score < semantic_similarity_threshold:
        return translate_dialect_to_english_arat5(dialect_text)
    else:
        return english_translation

In [28]:
semantic_similarity_threshold = 0.75
# calculate semantic similarity
def semantic_similarity(text1, text2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return similarity

In [29]:
def translate_dialect_to_english_unseen(dialect_text):
    helsinki_translation = translate_to_english_helsinki(dialect_text)
    # calculate semantic similarity with the translated English text
    similarity_score = semantic_similarity(helsinki_translation, dialect_text)
    # check if semantic similarity is below the threshold
    if similarity_score > semantic_similarity_threshold:
        print("arat5 used")
        return translate_dialect_to_english(dialect_text)
    else:
        return helsinki_translation

# second approach (using finetuned model)

In [30]:
arat5_weight = 0.15
helsinki_weight = 0.85
def translate_dialect_to_english_unseen(dialect_text):
    random_number = random.uniform(0, 1)
    if random_number < arat5_weight:
        print("AraT5 used")
        return translate_dialect_to_english(dialect_text)
    else:
        print("Helsinki used")
        return translate_to_english_helsinki(dialect_text)

In [31]:
arabic = data['egyption_Text'].tolist()
english=data['english_Text'].tolist()

In [32]:
from tqdm import tqdm

translations = []

# Iterate through Arabic texts and translate
for text in tqdm(arabic[:4000], desc="Translating"):
    translated_text = translate_dialect_to_english_unseen(text)
    translations.append(translated_text)



Translating:   0%|          | 0/4000 [00:00<?, ?it/s]

Helsinki used


Translating:   0%|          | 1/4000 [00:00<49:32,  1.35it/s]

Helsinki used


Translating:   0%|          | 2/4000 [00:01<45:47,  1.45it/s]

...

Translating: 100%|█████████▉| 3998/4000 [2:05:58<00:02,  1.47s/it]

AraT5 used


Translating: 100%|█████████▉| 3999/4000 [2:06:03<00:02,  2.27s/it]

Helsinki used


Translating: 100%|██████████| 4000/4000 [2:06:03<00:00,  1.89s/it]


In [33]:
translations

['The little prince.',
 'Antoine De Saint-Exuperi',
 'Chapter I',
 'Once, when I had six years ago, I saw a magnificent picture in a book on forests that no one had ever visited, the name of which was true stories.',
 'There was a fungus disease, a mushroom, and he was attacking a cheetah.',
 "Here's a copy of the drawing.",
 'The boa constrictors swallow their prey without chewing it, they said in the book.',
 "They can't move anymore and sleep for six months to digest.",
 'Then I thought a lot about the Adventures of the Jungle.',
 'I also succeeded in drawing his first drawing in wood colors.',
 'My drawing is Number One.',
 'She looked like this:',
 'I showed my masterpiece to the grown-ups and asked them if my drawing scared them.',
 'What should I fear in a hat, they replied.',
 'My drawing was not a hat drawing.',
 'It was a drawing that was tired of digesting an elephant.',
 'Then I decided to draw the boa tiredness from the inside, so the grown-ups could understand it.',
 'Bec

In [40]:
from nltk.translate.bleu_score import corpus_bleu

# Assuming english and translations are defined elsewhere
m = english[:4000]  # Take first 4000 sentences from english list
bleu_score = corpus_bleu([[m]], [translations])  # Wrap m and translations inside lists
print("BLEU score:", bleu_score)


BLEU score: 3.2397263153811994e-232
